In [35]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(42)

In [36]:
# Load the California Housing dataset
california_housing = fetch_california_housing()
data = california_housing.data
target = california_housing.target

In [37]:
california_housing.feature_names, len(california_housing.feature_names)

(['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 8)

In [38]:
train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.2, random_state=42)


In [39]:
# Initialize StandardScaler
scaler = StandardScaler()

# Fit and transform the training data
X_train_scaled = scaler.fit_transform(train_x)

# Transform the test data using the fitted scaler
X_test_scaled = scaler.transform(test_x)

# Find the minimum and maximum feature values of the last test data point
X_test_scaled[-1].min(), X_test_scaled[-1].max()


(-0.9211376319711084, 0.6044549279675977)

In [40]:
feature_x = torch.tensor(X_train_scaled, dtype=torch.float32)
test_feature_x = torch.tensor(X_test_scaled, dtype=torch.float32)
label = torch.tensor(train_y.reshape(-1,1), dtype=torch.float32)
test_label = torch.tensor(test_y.reshape(-1,1), dtype=torch.float32)


In [41]:
feature_x.shape, label.shape

(torch.Size([16512, 8]), torch.Size([16512, 1]))

In [51]:
class RegressionANN(nn.Module):
    def __init__(self, num_hidden):
        super(RegressionANN, self).__init__()
        # Set the random seed
        torch.manual_seed(42)
        self.layer1 = nn.Linear(8,num_hidden)
        self.layer2 = nn.Linear(num_hidden,1)
        self.num_hidden = num_hidden
    
    def forward(self, data_point):
        activation1 = F.relu(self.layer1(data_point))
        output = self.layer2(activation1)
        return output


In [67]:
model = RegressionANN(num_hidden=16)

In [68]:
print(model)

RegressionANN(
  (layer1): Linear(in_features=8, out_features=16, bias=True)
  (layer2): Linear(in_features=16, out_features=1, bias=True)
)


In [69]:
model.layer2.bias.data.item()
# 0.22723788022994995

0.22723788022994995

In [70]:
model.layer1.bias.data

tensor([ 0.2964, -0.0702,  0.3042,  0.1102, -0.2994,  0.2447, -0.0973, -0.1355,
        -0.2935, -0.3515,  0.1012, -0.0772,  0.1376, -0.2901,  0.2625, -0.2595])

In [71]:
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

In [72]:
# Print optimizer object to see its attributes
print(optimizer)

# Get optimizer state dictionary
optimizer_state_dict = optimizer.state_dict()
print(optimizer_state_dict)

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.01
    weight_decay: 0
)
{'state': {}, 'param_groups': [{'lr': 0.01, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False, 'params': [0, 1, 2, 3]}]}


In [57]:
# output = model(feature_x)

# loss_function = loss_func(output, label)

# loss_function.item()
# # 4.202902793884277

In [73]:
epochs = 100

for epoch in range(epochs):
    # zero grad
    optimizer.zero_grad()

    # forward pass
    output = model(feature_x)

    # compute loss
    loss = loss_func(output, label)

    # back_prop
    loss.backward()

    # update parameter
    optimizer.step()
    # print loss every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")


#  hidden 16 -> Loss: 0.4750
#  hidden 64 -> Loss: 6.6225
loss.item()

Epoch [10/100], Loss: 2.3334
Epoch [20/100], Loss: 1.1195
Epoch [30/100], Loss: 0.8087
Epoch [40/100], Loss: 0.7287
Epoch [50/100], Loss: 0.6495
Epoch [60/100], Loss: 0.6001
Epoch [70/100], Loss: 0.5567
Epoch [80/100], Loss: 0.5241
Epoch [90/100], Loss: 0.4970
Epoch [100/100], Loss: 0.4750


0.47498172521591187

In [62]:
#  64 neuron in hidden layer
model64 = RegressionANN(num_hidden=64)
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model64.parameters(), lr = 0.01)
for _ in range(100):
        optimizer.zero_grad()
        y_pred = model64(feature_x)

        loss = loss_fn(y_pred, label)
        loss.backward()
        optimizer.step()
loss.item()

0.4150702953338623